In [1]:
from importlib import reload
import os
from toolz import pipe as p

import scrapDataHelpers

In [26]:
ls image_data/

gossiping/  laughing/     pullinghair/  quarrel/   stabbing/
isolation/  nonbullying/  punching/     slapping/  strangle/


In [29]:
sum(scrapDataHelpers.classPercentages('image_data')['count'].values())

2522

In [31]:
2000/2522

0.7930214115781126

In [32]:
len([f for d in os.listdir('image_data') for 
     f in p(os.path.join('image_data', d), os.listdir)])

n_train = 2000
n_val = None

dest_dir = 'scrap_data' + str(n_train)

In [33]:
reload(scrapDataHelpers)

<module 'scrapDataHelpers' from '/home/john/projects/dl/scrapDataHelpers.py'>

In [34]:
classes = os.listdir('image_data')
scrapDataHelpers.makeScrapData(classes, n_train=n_train, n_val=n_val)

In [35]:
scrapDataHelpers.classPercentages(dest_dir)

{'percent': {('train', 'strangle'): 0.14486215538847116,
  ('val', 'strangle'): 0.1442125237191651,
  ('train', 'quarrel'): 0.09924812030075188,
  ('val', 'quarrel'): 0.09867172675521822,
  ('train', 'gossiping'): 0.16390977443609023,
  ('val', 'gossiping'): 0.16318785578747627,
  ('train', 'pullinghair'): 0.10325814536340852,
  ('val', 'pullinghair'): 0.10436432637571158,
  ('train', 'nonbullying'): 0.039598997493734335,
  ('val', 'nonbullying'): 0.03984819734345351,
  ('train', 'isolation'): 0.10025062656641603,
  ('val', 'isolation'): 0.10056925996204934,
  ('train', 'slapping'): 0.08471177944862156,
  ('val', 'slapping'): 0.08538899430740038,
  ('train', 'laughing'): 0.06265664160401002,
  ('val', 'laughing'): 0.06261859582542695,
  ('train', 'punching'): 0.14536340852130325,
  ('val', 'punching'): 0.1442125237191651,
  ('train', 'stabbing'): 0.056140350877192984,
  ('val', 'stabbing'): 0.056925996204933584},
 'count': {('train', 'strangle'): 289,
  ('val', 'strangle'): 76,
  ('tra

# Just try 2 classes

In [93]:
scrapDataHelpers.makeScrapData(['laughing', 'punching'], n_train=50, n_val=10)

In [94]:
scrapDataHelpers.classPercentages('scrap_data50/')

({('train', 'laughing'): 0.3015267175572519,
  ('val', 'laughing'): 0.29411764705882354,
  ('train', 'punching'): 0.6984732824427481,
  ('val', 'punching'): 0.7058823529411765},
 {('train', 'laughing'): 79,
  ('val', 'laughing'): 15,
  ('train', 'punching'): 183,
  ('val', 'punching'): 36})

In [95]:
scrapDataHelpers.classPercentages('image_data', classes = ['laughing', 'punching'])

({'laughing': 158, 'punching': 366},
 {'laughing': 30.15267175572519, 'punching': 69.8473282442748})